In [1]:
# import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings

warnings.filterwarnings("ignore")

### data loading and intial inspection

In [2]:
data = pd.read_csv(r"C:\Users\asant\OneDrive\Desktop\retail banking\data\bank_data_C.csv")

In [3]:
data.head(10)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.00
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.00
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.00
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.00
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.50
5,T6,C1536588,8/10/72,F,ITANAGAR,53609.20,2/8/16,173940,676.00
6,T7,C7126560,26/1/92,F,MUMBAI,973.46,2/8/16,173806,566.00
7,T8,C1220223,27/1/82,M,MUMBAI,95075.54,2/8/16,170537,148.00
8,T9,C8536061,19/4/88,F,GURGAON,14906.96,2/8/16,192825,833.00
9,T10,C6638934,22/6/84,M,MUMBAI,4279.22,2/8/16,192446,289.11


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041614 entries, 0 to 1041613
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1041614 non-null  object 
 1   CustomerID               1041614 non-null  object 
 2   CustomerDOB              1041614 non-null  object 
 3   CustGender               1041614 non-null  object 
 4   CustLocation             1041614 non-null  object 
 5   CustAccountBalance       1041614 non-null  float64
 6   TransactionDate          1041614 non-null  object 
 7   TransactionTime          1041614 non-null  int64  
 8   TransactionAmount (INR)  1041614 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 71.5+ MB


In [5]:
data.describe(include="all")

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
count,1041614,1041614,1041614,1041614,1041614,1.041614e+06,1041614,1.041614e+06,1.041614e+06
unique,1041614,879358,17233,3,9275,NaN,55,NaN,NaN
top,T1,C5533885,1/1/1800,M,MUMBAI,NaN,7/8/16,NaN,NaN
freq,1,6,56292,760978,101997,NaN,27084,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,1.149986e+05,NaN,1.571221e+05,1.566096e+03
std,NaN,NaN,NaN,NaN,NaN,8.467609e+05,NaN,5.126352e+04,6.561464e+03
min,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,4.728140e+03,NaN,1.240410e+05,1.600000e+02
50%,NaN,NaN,NaN,NaN,NaN,1.676852e+04,NaN,1.642470e+05,4.575000e+02
75%,NaN,NaN,NaN,NaN,NaN,5.742885e+04,NaN,2.000220e+05,1.200000e+03


### Data cleaning

In [6]:
data.isna().sum()

TransactionID              0
CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64

### rename columns

In [7]:
data = data.rename(columns={"TransactionAmount (INR)": "TransactionAmount"})
list(data.columns)

['TransactionID',
 'CustomerID',
 'CustomerDOB',
 'CustGender',
 'CustLocation',
 'CustAccountBalance',
 'TransactionDate',
 'TransactionTime',
 'TransactionAmount']

### parse dates

In [11]:
def normalize_dmy(x):
    """Normalize day-first dates to DD/MM/YY format."""
    if pd.isna(x): return np.nan
    try:
            d, m, y = str(x).strip().split('/')
            return f"{int(d):02d}/{int(m):02d}/{int(y):02d}"
    except:
            return np.nan

for col in ["CustomerDOB", "TransactionDate"]:
    data[col] = data[col].map(normalize_dmy)
    data[col] = pd.to_datetime(data[col], format="%d/%m/%y", errors="coerce")

data[["CustomerDOB", "TransactionDate"]].head()

,CustomerDOB,TransactionDate
0,1994-01-10,2016-08-02
1,2057-04-04,2016-08-02
2,1996-11-26,2016-08-02
3,1973-09-14,2016-08-02
4,1988-03-24,2016-08-02


### fix dates of birth
#### fix: subtract 100 years from DOB> TODAY 

In [12]:
from pandas.tseries.offsets import DateOffset

mask = data["CustomerDOB"] > pd.Timestamp.today()
data.loc[mask, "CustomerDOB"]-= DateOffset(years=100)
print(f"Fixed {mask.sum()} future DOBs")
data[["CustomerDOB"]].head(10)

Fixed 54111 future DOBs


,CustomerDOB
0,1994-01-10
1,1957-04-04
2,1996-11-26
3,1973-09-14
4,1988-03-24
5,1972-10-08
6,1992-01-26
7,1982-01-27
8,1988-04-19
9,1984-06-22


In [13]:
data.dropna(inplace = True)

In [15]:
data.isna().sum()

TransactionID          0
CustomerID             0
CustomerDOB            0
CustGender             0
CustLocation           0
CustAccountBalance     0
TransactionDate        0
TransactionTime        0
TransactionAmount      0
TransactionDatetime    0
dtype: int64

### parse traction time
#### why: combine date + time for accurate analysis(eg. hourly patterns)

In [14]:
# Make sure both columns are strings and pad time values to 6 digits
tt = data["TransactionTime"].astype(str).str.zfill(6)

# Combine date and time into one datetime column
data["TransactionDatetime"] = pd.to_datetime(
    data["TransactionDate"].astype(str) + " " + tt,
    format="%Y-%m-%d %H%M%S", errors="coerce"
)

# Check for any parsing errors
print(f"Datetime nulls: {data['TransactionDatetime'].isna().sum()}")

# View the first few rows
data[["TransactionDate", "TransactionTime", "TransactionDatetime"]].head()


Datetime nulls: 0


,TransactionDate,TransactionTime,TransactionDatetime
0,2016-08-02,143207,2016-08-02 14:32:07
1,2016-08-02,141858,2016-08-02 14:18:58
2,2016-08-02,142712,2016-08-02 14:27:12
3,2016-08-02,142714,2016-08-02 14:27:14
4,2016-08-02,181156,2016-08-02 18:11:56


### DUPLICATE CHECK

In [17]:
# Count exact duplicate rows
exact_dups = data.duplicated().sum()

# Count duplicate Transaction IDs
txid_dups = data["TransactionID"].duplicated().sum()

# Print results
print(f"Exact duplicates: {exact_dups}")
print(f"Duplicate TransactionIDs: {txid_dups}")


Exact duplicates: 0
Duplicate TransactionIDs: 0
